# Vertex AI Search - Technical Deep Dive - Lab Exercise

The purpose of this lab is to explore the use of the client libraries and APIs in Vertex AI Search and the Langchain LLM integrations and retrievers for Enterprise Search and Vertex AI.

You'll use these tools to build a question and answer service that takes a user query, retrieves relevant documents from a search data store in Gen App Builder, then returns an LLM-generated answer to the original query along with source documents that were used to generate the answer.

Helpful resources for the lab coding exercise:

- [Vertex AI Search Code Samples (Documentation)](https://cloud.google.com/generative-ai-app-builder/docs/samples)
- [Question Answering Over Documents (GitHub)](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gen-app-builder/retrieval-augmented-generation/examples/question_answering.ipynb)
- [Grounding Generative AI using Vertex AI Search Results (Colab)](https://colab.research.google.com/drive/174YYPNNy1rWdIFvV-_LWZ-cueRB7Q6EC?resourcekey=0-9bYTUjXMbEkHIuduaNjNJw&usp=sharing)
- [Vertex AI Search - Search Web App (GitHub)](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gen-app-builder/search-web-app)

# Coding exercise (Technical asset)

## Step 1

As a first step, you must create an unstructured data search app that uses PDFs data and get de data_store_id that will be used later.


**DONE?** :)

## Step 2

Install the Vertex AI and Langchain 0.0.236 (newer versions are broken as of 2023-08-10) client libraries for Python:

In [18]:
# Install packages
# Note: You might need to restart the runtime after installing these packages
#!pip install google-cloud-discoveryengine google-cloud-aiplatform langchain==0.0.236 "shapely<2.0.0" -q
!pip install google-cloud-discoveryengine google-cloud-aiplatform langchain -q

In [4]:
!pip install gradio==3.48.0 -q

Next code will restart your Runtime



In [26]:
import os
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Killing tunnel 127.0.0.1:7860 <> https://aa45d9ef32a90f9b32.gradio.live


## Step 3

Use the [Vertex AI Search document retriever in LangChain](https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search) to retrieve documents from your data store based on a query.


Here we introduce the project ID where we will run the cloud services, the location and Data Store ID previously created.

In [ ]:
# @title Configuration Variables and Test Retriever
PROJECT_ID = "qwiklabs-gcp-02-95597416198e" # @param {type:"string"}
LOCATION = "us-central1" # @param {type:"string"}
DATA_STORE_ID = "orange-pdf_1702533306233" # @param {type:"string"}

And start the execution

In [2]:
# Authenticate with Google account
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [3]:
from os.path import basename
from typing import Dict, List, Optional, Tuple, Any

In [24]:
# from google.cloud import discoveryengine as discoveryengine
from langchain.retrievers import (
    GoogleVertexAISearchRetriever
)
QUERY = "Quien es el CEO de Orange?"

# Code your solution here

# Initialise an Vertex AI Search Retriever
retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    search_engine_id=DATA_STORE_ID
    #,max_documents=3 #opt
    #,max_extractive_answer_count=3, #opt
    ,get_extractive_answers=True #opt
    )
# Get relevant documents
result = retriever.get_relevant_documents(QUERY)
for doc in result:
    print(doc)


page_content='FY22 results\n\n9\n\nEBITDAaL\nreaching\n€13bn in line\nwith guidance\n\n9\n\nGroup\nEBITDAaL\n\nyoy\n+8.5%\n+€269m\n\nyoy\n+2.5%\n+€318m\n\n92\n\n262\n\n19\n\n13\n\n141\n\nSpain\n\n12,963\n\n25\n\nFY 21 cb France\n\nTotem\n\nMobile\nFinancial\nServices\n\nEnterprise\n\nFY 22\n\n-186\n\nOther\n\nMEA\n\nICSS**\n\nEuropeans\ncountries\n\n-47\n\n12,645\n\nFY 2022 Group EBITDAaL evolution per segment (yoy, in€m)\n\nTelecom*\nEBITDAaL\n\nyoy\n+7.9%\n+€256m\n\nyoy\n+2.4%\n+€306m\n\n+0.4%\n\n-4.0%\n\n-18.8%\n\n+5.8%\n\n+5.4%\n\n* The Mobile Financial Services business segment includes the activities of Orange Bank and Orange Bank Africa (with Orange Money business reported under MEA segment).\n** International CarrIers and Shared Services\n\nQ4 2022\n\nFY 2022\n\n+11.3%\n\n+10.0%\n\n+2.5%\n\n+59.7%' metadata={'id': 'd8d4ab56eae856a1156c2b83a91c456e', 'source': 'gs://jrdetorre-orange-demo/Orange-Q4 2022 Financial Results.pdf'}


## Step 4

Given a search query, use [Langchain's LLM integration with Vertex AI](https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm) to send a search query and return an answer with source documents

Hint: Use [RetrievalQAWithSourcesChain](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gen-app-builder/retrieval-augmented-generation/examples/question_answering.ipynb) and refer to the “Helpful resources” at the top of this notebook!

Sample query: “Who is the CEO of DeepMind?”

In [6]:
import vertexai
from langchain.llms import VertexAI
#from langchain.retrievers import GoogleVertexAISearchRetriever

# Code your solution here
from langchain.chains import LLMChain
from langchain import PromptTemplate
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

Here we define the parameters for LLM model calls

In [25]:
#Initialise LLM
LLM_MODEL = "text-bison@001" #@param {type: "string"}
MAX_OUTPUT_TOKENS = 1024 #@param {type: "integer"}
TEMPERATURE = 0.2 #@param {type: "number"}
TOP_P = 0.8 #@param {type: "number"}
TOP_K = 40 #@param {type: "number"}
VERBOSE = True #@param {type: "boolean"}
llm_params = dict(
    model_name=LLM_MODEL,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    top_k=TOP_K,
    verbose=VERBOSE,
)

llm = VertexAI(**llm_params)

Sample question for testing

In [26]:
QUERY = "Quien es el CEO de Orange?" #@param {type: "string"}

Here we test the results before productivicing


In [27]:
# Combine the LLM with a prompt to make a simple chain
PROMPT_STRING = "Please parse these search results and summarize them to the answer the following question. Results:{results}. Question:{query}. Answer:"
prompt = PromptTemplate(input_variables=['results', 'query'],
                        template=PROMPT_STRING)
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

# Get relevant documents
documents = retriever.get_relevant_documents(QUERY)
content = [d.page_content for d in documents]

# Use the LLM-prompt chain to answer the question based on the results
result = chain.run({'results': content, 'query': QUERY})

result.split('\n')



> Entering new LLMChain chain...
Prompt after formatting:
Please parse these search results and summarize them to the answer the following question. Results:['FY22 results\n\n9\n\nEBITDAaL\nreaching\n€13bn in line\nwith guidance\n\n9\n\nGroup\nEBITDAaL\n\nyoy\n+8.5%\n+€269m\n\nyoy\n+2.5%\n+€318m\n\n92\n\n262\n\n19\n\n13\n\n141\n\nSpain\n\n12,963\n\n25\n\nFY 21 cb France\n\nTotem\n\nMobile\nFinancial\nServices\n\nEnterprise\n\nFY 22\n\n-186\n\nOther\n\nMEA\n\nICSS**\n\nEuropeans\ncountries\n\n-47\n\n12,645\n\nFY 2022 Group EBITDAaL evolution per segment (yoy, in€m)\n\nTelecom*\nEBITDAaL\n\nyoy\n+7.9%\n+€256m\n\nyoy\n+2.4%\n+€306m\n\n+0.4%\n\n-4.0%\n\n-18.8%\n\n+5.8%\n\n+5.4%\n\n* The Mobile Financial Services business segment includes the activities of Orange Bank and Orange Bank Africa (with Orange Money business reported under MEA segment).\n** International CarrIers and Shared Services\n\nQ4 2022\n\nFY 2022\n\n+11.3%\n\n+10.0%\n\n+2.5%\n\n+59.7%']. Question:Quien es el CEO de Orang

['The CEO of Orange is Stéphane Richard.']

In [28]:
content

['FY22 results\n\n9\n\nEBITDAaL\nreaching\n€13bn in line\nwith guidance\n\n9\n\nGroup\nEBITDAaL\n\nyoy\n+8.5%\n+€269m\n\nyoy\n+2.5%\n+€318m\n\n92\n\n262\n\n19\n\n13\n\n141\n\nSpain\n\n12,963\n\n25\n\nFY 21 cb France\n\nTotem\n\nMobile\nFinancial\nServices\n\nEnterprise\n\nFY 22\n\n-186\n\nOther\n\nMEA\n\nICSS**\n\nEuropeans\ncountries\n\n-47\n\n12,645\n\nFY 2022 Group EBITDAaL evolution per segment (yoy, in€m)\n\nTelecom*\nEBITDAaL\n\nyoy\n+7.9%\n+€256m\n\nyoy\n+2.4%\n+€306m\n\n+0.4%\n\n-4.0%\n\n-18.8%\n\n+5.8%\n\n+5.4%\n\n* The Mobile Financial Services business segment includes the activities of Orange Bank and Orange Bank Africa (with Orange Money business reported under MEA segment).\n** International CarrIers and Shared Services\n\nQ4 2022\n\nFY 2022\n\n+11.3%\n\n+10.0%\n\n+2.5%\n\n+59.7%']

Here we create a function that will be called from Gradio

In [23]:
def data_store_qna(QUERY):

  PROMPT_STRING = "Please parse these search results and summarize them to the answer the following question. Results:{results}. Question:{query}. Answer:"
  prompt = PromptTemplate(input_variables=['results', 'query'],
                        template=PROMPT_STRING)
  chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

# Get relevant documents
  documents = retriever.get_relevant_documents(QUERY)
  content = [d.page_content for d in documents]

# Use the LLM-prompt chain to answer the question based on the results
  result = chain.run({'results': content, 'query': QUERY})

  result.split('\n')
  return result

Testing the function

In [17]:
output=data_store_qna(QUERY)
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
Please parse these search results and summarize them to the answer the following question. Results:['FY22 results\n\n9\n\nEBITDAaL\nreaching\n€13bn in line\nwith guidance\n\n9\n\nGroup\nEBITDAaL\n\nyoy\n+8.5%\n+€269m\n\nyoy\n+2.5%\n+€318m\n\n92\n\n262\n\n19\n\n13\n\n141\n\nSpain\n\n12,963\n\n25\n\nFY 21 cb France\n\nTotem\n\nMobile\nFinancial\nServices\n\nEnterprise\n\nFY 22\n\n-186\n\nOther\n\nMEA\n\nICSS**\n\nEuropeans\ncountries\n\n-47\n\n12,645\n\nFY 2022 Group EBITDAaL evolution per segment (yoy, in€m)\n\nTelecom*\nEBITDAaL\n\nyoy\n+7.9%\n+€256m\n\nyoy\n+2.4%\n+€306m\n\n+0.4%\n\n-4.0%\n\n-18.8%\n\n+5.8%\n\n+5.4%\n\n* The Mobile Financial Services business segment includes the activities of Orange Bank and Orange Bank Africa (with Orange Money business reported under MEA segment).\n** International CarrIers and Shared Services\n\nQ4 2022\n\nFY 2022\n\n+11.3%\n\n+10.0%\n\n+2.5%\n\n+59.7%']. Question:Quien es el CEO de Orang

We import Gradio and create a simple website. It will provide us with an URL that we can use to test it.

In [29]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Ask Vertex AI Search

    This app uses Vertex AI Search Results to answer questions.

    ## How to use

    1. Enter a question
    2. Click on "Ask the Question"
    3. The answer will be displayed

    """)
    with gr.Row():
      with gr.Column():
        input_text = gr.Textbox(label="Question", placeholder="Quién es el CEO de Orange?")
    with gr.Row():
      generate = gr.Button("Ask the Question")
    with gr.Row():
      label = gr.Textbox(label="Output")
    generate.click(data_store_qna,input_text, [label])
demo.launch(share=True, debug=False)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b4782df23e582faf97.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
